In [1]:
import os
import socket
import torch
import torch.distributed as dist

from coral.configs.reward_model.config_loader import (
    load_reward_model_config,
    update_config,
)
from coral.training.reward.setup import (
    training_init,
    build_trainer,
)

from coral.distributed.utils.activation_ckpt import apply_fsdp_checkpointing

/home/omkar/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import os
import torch.distributed as dist
from torch.distributed.algorithms._checkpoint.checkpoint_wrapper import (
    checkpoint_wrapper,
    CheckpointImpl,
    apply_activation_checkpointing,
)
from transformers.models.qwen3.modeling_qwen3 import Qwen3DecoderLayer
from functools import partial

torch.backends.cuda.enable_flash_sdp(True)
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_math_sdp(False)

# non_reentrant_wrapper = partial(
#     checkpoint_wrapper,
#     offload_to_cpu=False,
#     checkpoint_impl=CheckpointImpl.NO_REENTRANT,
# )

# check_fn = lambda submodule: isinstance(submodule, Qwen3DecoderLayer)


# def apply_fsdp_checkpointing(model):
#     """apply activation checkpointing to model
#     returns None as model is updated directly
#     """
#     print(f"--> applying fdsp activation checkpointing...")

#     apply_activation_checkpointing(
#         model, checkpoint_wrapper_fn=non_reentrant_wrapper, check_fn=check_fn
#     )

device = torch.device('cuda')
cfg = load_reward_model_config("coral/configs/reward_model.yaml")

# ---- init training ----
model, tokenizer, loader, optimizer, scheduler = training_init(
    cfg,
    device=device,
)
apply_fsdp_checkpointing(model)
model.encoder.backbone.config.use_cache = False
model.encoder.backbone.gradient_checkpointing_disable()  
model.encoder.backbone.config.attn_implementation = "sdpa"


trainer = build_trainer(
    model=model,
    tokenizer=tokenizer,
    loader=loader,
    optimizer=optimizer,
    scheduler=scheduler,
    cfg=cfg,
    device=device
)


start_step = 0

for step, batch in enumerate(loader):
    if step < start_step:
        continue

    metrics = trainer.step(batch)

    print(metrics)


Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/omkar/.cache/huggingface/hub/models--hyp1231--blair-roberta-large/.no_exist/35aef55f2aa61d0b22f52df794ebe53fa25cb228/adapter_config.json'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/omkar/.cache/huggingface/hub/models--hyp1231--blair-roberta-large/.no_exist/35aef55f2aa61d0b22f52df794ebe53fa25cb228/adapter_config.json'
`torch_dtype` is deprecated! Use `dtype` instead!
Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/omkar/.cache/huggingface/hub/models--hyp1231--blair-roberta-large/.no_exist/35aef55f2aa61d0b22f52df794ebe53fa25cb228/adapter_config.json'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/omka

[DoRA] arch=roberta | layers 0..23
[optimizer] group 'encoder': 432 params | lr=0.0001
[optimizer] group 'head': 6 params | lr=0.0001
--> applying fdsp activation checkpointing...


/mnt/nfs/share/code/coral/coral/training/reward/trainer.py:49: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
/mnt/nfs/share/code/coral/coral/training/reward/process.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels[i, :n_i] = torch.tensor(labels_i, dtype=torch.long)
/mnt/nfs/share/code/coral/coral/training/reward/trainer.py:154: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=True, dtype=torch.bfloat16):


{'loss': 0.3649566173553467, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
{'loss': 0.3635106086730957, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
{'loss': 0.36209049820899963, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
{'loss': 0.3628426790237427, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
{'loss': 0.36805620789527893, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
{'loss': 0.36508849263191223, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
{'loss': 0.3638114631175995, 'gamma': 1.05, 'stepped': False, 'global_step': 0}
step 1 | loss 0.3684 | ±1 acc 0.712 | exact acc 0.159 | gamma 1.05
{'loss': 0.36841464042663574, 'gamma': 1.05, 'stepped': True, 'global_step': 1}
{'loss': 0.36551591753959656, 'gamma': 1.05, 'stepped': False, 'global_step': 1}
{'loss': 0.3675881326198578, 'gamma': 1.05, 'stepped': False, 'global_step': 1}
{'loss': 0.36572542786598206, 'gamma': 1.05, 'stepped': False, 'global_step': 1}
{'loss': 0.36413004994392395, 'gamma': 1.05, 'st

In [ ]:
from coral.models.reward.utils import get_transformer_layers

In [7]:
type(get_transformer_layers(model.encoder)[0])

transformers.models.roberta.modeling_roberta.RobertaLayer